In [19]:
import math
import pandas as pd
import matplotlib.pyplot as plt
import kaleido
pd.options.plotting.backend = "plotly"
import plotly.express as px
import plotly.graph_objects as go



In [23]:
# Psi function + costs for ex-post degradation calculation
def psi(A,B,SOC):
    return A* ((1-SOC)**B)

CAPEX = 95000

deg_sh = 0.1355593607305936

In [24]:
# Ex-post degradation calculation
results = pd.read_csv('../model/all_markets_results_new.csv')
SOC = results[['SOC_BESS']].copy()
SOC['psi'] = SOC.apply(lambda x: psi(0.000274,2.1,x['SOC_BESS']),axis=1)
SOC['delta_psi_cost'] = abs(SOC['psi'].diff()) * CAPEX /2
SOC.apply(lambda x: x['delta_psi_cost'] if x['delta_psi_cost'] > deg_sh else deg_sh,axis=1).sum()

np.float64(28.156083470527385)

In [ ]:
# Operation plot
capacity_results = results[['aFRR_pos','aFRR_neg','PCR']].copy().ffill()
color_FCR_pos = '#6f6fff'
color_FCR_neg = '#5757c9'
color_aFRR_pos = '#f19494'
color_aFRR_neg = '#b14646'
color_op = '#000000'

opacities = [1,1,1] #FCR, aFRR, SOC

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(capacity_results.index),
    y=results['SOC_BESS'].ffill(),
    mode='lines',
    name='SOC',
    line=dict(color=color_op, width=2),
    opacity=opacities[2],
))

# Add the first dataset
fig.add_trace(go.Bar(
    x=list(capacity_results.index),
    y=capacity_results['PCR']/2,
    name='PCR/2',
    marker_color=color_FCR_pos,
    opacity=opacities[0],
))

# Add the first dataset
fig.add_trace(go.Bar(
    x=list(capacity_results.index),
    y=capacity_results['aFRR_pos'],
    name='aFRR_pos',
    marker_color = color_aFRR_pos,
    opacity=opacities[1],
))

# Add the second dataset with transparency
fig.add_trace(go.Bar(
    x=list(capacity_results.index),
    y=1-capacity_results['aFRR_neg']-capacity_results['PCR'] - capacity_results['aFRR_pos'],
    name='free',
    showlegend=False,
    opacity=0
))

# Add the first dataset
fig.add_trace(go.Bar(
    x=list(capacity_results.index),
    y=capacity_results['aFRR_neg'],
    name='aFRR_neg',
    marker_color = color_aFRR_neg,
    opacity=opacities[1],
))

# Add the first dataset
fig.add_trace(go.Bar(
    x=list(capacity_results.index),
    y=capacity_results['PCR']/2,
    name='PCR/2',
    marker_color=color_FCR_neg,
    opacity=opacities[0],
))


# Update layout for stacked bars
fig.update_layout(barmode='stack')
fig.update_layout(bargap=0)
fig.update_traces(marker=dict(line=dict(width=0)))

print(results['SOC_BESS'])

# Convert the x axis to datetime and set tick interval to every 2 hours
fig.update_xaxes(
    tickvals=results.index[::8],  # Assuming data is in 15-minute intervals, every 8th tick is 2 hours
    ticktext=[pd.to_datetime(t).strftime('%H:%M') for t in results['Unnamed: 0'][::8]],
    tickangle=45  # Rotate tick text vertically
)

# Show the figure
fig.show()
#fig.to_image("out_test.png",format="png", engine="kaleido")

aaaa
0     0.500000
1     0.500000
2     0.499999
3     0.500000
4     0.500000
        ...   
91    0.500000
92    0.500000
93    0.500000
94    0.500000
95    0.500000
Name: SOC_BESS, Length: 96, dtype: float64
